In [1]:
!nvidia-smi

Sun Nov  7 16:48:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:39:00.0 Off |                    0 |
| N/A   36C    P0    57W / 312W |  27838MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [16]:
!python ddp_example.py --batch_size 16

2021-11-07 17:27:28.760154: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Epoch 0
000: Loss 10.1966
020: Loss 9.0277
040: Loss 9.1042
060: Loss 9.1736
080: Loss 9.2037
100: Loss 8.8683
^C


In [ ]:
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP

In [ ]:
torch.manual_seed(42)

In [ ]:
# you need this argument in your scripts for DDP to work
parser.add_argument('--local_rank', type=int, default=-1, metavar='N', help='Local process rank.')  

In [ ]:
# initialize PyTorch distributed using environment variables (you could also do this more explicitly by specifying `rank` and `world_size`, but I find using environment variables makes it so that you can easily use the same script on different machines)
dist.init_process_group(backend='nccl', init_method='env://')
torch.cuda.set_device(args.local_rank)

In [ ]:
sampler = DistributedSampler(dataset)

In [ ]:
# initialize distributed data parallel (DDP)
model = DDP(
    model,
    device_ids=[args.local_rank],
    output_device=args.local_rank
)

In [ ]:
# let all processes sync up before starting with a new epoch of training
dist.barrier()

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=4 --nnodes=1 --node_rank 0 ddp_example2.py --batch_size 16